In [2]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
from keras import regularizers 
import keras.backend as K

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from lifelines.datasets import load_rossi
from sklearn.model_selection import train_test_split
from sklearn import preprocessing



Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [9]:
pip show pandas

Name: pandas
Version: 0.24.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: http://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/site-packages
Requires: numpy, pytz, python-dateutil
Required-by: statsmodels, sklearn-pandas, shap, seaborn, scikit-survival, random-survival-forest, pysurvival, pgmpy, mlflow, lifelines
Note: you may need to restart the kernel to use updated packages.


In [3]:
X_train = pd.read_csv('/Users/zvisegal/Desktop/patient2rank/data/x_train.csv')
X_validation = pd.read_csv('/Users/zvisegal/Desktop/patient2rank/data/x_validation.csv')
X_test = pd.read_csv('/Users/zvisegal/Desktop/patient2rank/data/x_test.csv')
X_train = X_train.set_index('patient_id', drop=True)
X_validation = X_validation.set_index('patient_id', drop=True)
X_test = X_test.set_index('patient_id', drop=True)
X=pd.concat([X_train,X_test,X_validation], ignore_index=True)

y = pd.concat([X_train.target, X_validation.target, X_test.target])

y.rename('target')

y_survival = pd.read_csv('/Users/zvisegal/Desktop/patient2rank/data/y_survival.csv')
y_survival = y_survival.set_index('patient_id')
y_survival = y_survival[['target', 'time']]
y_survival.target = y_survival.target.astype(bool)
y_survival.time = y_survival.time.astype('double')

y_survival = y_survival.to_records(index=False)
Y=y_survival
# global_time = '2020-03-10 15:23:35'

In [7]:
y_survival

rec.array([(False, 692.), (False, 656.), (False, 704.), ( True, 402.),
           (False, 690.), (False, 652.), (False, 624.), (False, 666.),
           (False, 704.), (False, 692.), (False, 669.), (False, 698.),
           ( True, 152.), (False, 654.), (False, 695.), ( True, 416.),
           (False, 620.), ( True, 315.), (False, 699.), ( True, 517.),
           (False, 693.), (False, 705.), (False, 654.), (False, 624.),
           (False, 664.), (False, 615.), ( True, 553.), (False, 678.),
           (False, 640.), (False, 690.), (False, 705.), (False, 687.),
           (False, 646.), (False, 686.), (False, 629.), ( True, 181.),
           (False, 672.), ( True, 179.), ( True, 407.), (False, 670.),
           (False, 644.), ( True, 516.), ( True,  93.), (False, 651.),
           (False, 698.), (False, 662.), ( True, 145.), (False, 639.),
           (False, 684.), (False, 630.), (False, 682.), (False, 664.),
           (False, 682.), ( True, 435.), (False, 648.), ( True, 494.),
      

In [92]:
E=y_survival.target.astype(int)
Y=y_survival.time

In [93]:
X=X.drop([ 'trial_start_time',
       'max_time', 'eligability_end_date', 'inclusion', 'exclusion',
       'target', 'target_diagnosis_time', 'target_code', 'target_name',
       'target_type', 'exclusion_diagnosis_time', 'exclusion_code',
       'inclusion_diagnosis_time', 'inclusion_code', 'duration_pre_diag'],axis=1)


In [100]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.25, random_state=0)
X_train,X_val,E_train,E_val=train_test_split(X,E,test_size=0.25, random_state=0)
#Standardize
scaler=preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
# X_val=scaler.transform(X_val)

In [102]:
X_train

array([[-0.95437311,  1.03035687, -0.77735325, ..., -0.08732634,
        -0.28743396, -0.13177786],
       [ 1.04780824,  0.27902793,  1.28641644, ...,  0.48416164,
         1.45452668,  0.41265625],
       [-0.95437311, -0.09663653,  1.28641644, ..., -0.08732634,
        -0.28743396, -0.13177786],
       ...,
       [-0.95437311, -0.75411364, -0.77735325, ..., -0.08732634,
        -0.28743396, -0.13177786],
       [ 1.04780824, -0.37844917, -0.77735325, ..., -0.08732634,
        -0.28743396, -0.13177786],
       [ 1.04780824,  0.6546924 , -0.77735325, ..., -0.08732634,
        -0.28743396, -0.13177786]])

In [103]:

#Sorting for NNL!
sort_idx = np.argsort(Y_train)[::-1]
X_train=X_train[sort_idx]
Y_train=Y_train[sort_idx]
E_train=E_train[sort_idx]


Pipline

In [104]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

input_shape=X_train.shape[1]
scaler=preprocessing.StandardScaler()
minmax_scaler = MinMaxScaler()
pipeline = Pipeline([('scaler', scaler)])
pipeline.steps.append(('normalize', minmax_scaler))

In [105]:
pipeline

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('normalize', MinMaxScaler(copy=True, feature_range=(0, 1)))],
         verbose=False)

CALLBACKS

In [106]:
from keras import callbacks
import time

name=f'deepsurv_opioid{int(time.time())}'
tensoboard=callbacks.TensorBoard(
    log_dir=f'deep_surv_logs/{name}')


In [63]:
import tensorflow as tf
import datetime, os

%load_ext tensorboard
%tensorboard --logdir=deep_surv_logs

MODEL

In [107]:
def negative_log_likelihood(E):
    def loss(y_true, y_pred):
        hazard_ratio = K.exp(y_pred)
        log_risk = K.log(K.cumsum(hazard_ratio, axis=1))
        uncensored_likelihood = y_pred - log_risk
        censored_likelihood = uncensored_likelihood * E
        neg_likelihood = -K.sum(censored_likelihood)
        return neg_likelihood
    return loss

#Keras model
model = Sequential()
model.add(Dense(64, input_shape=(input_shape,), init='glorot_uniform')) # shape= length, dimension
model.add(Activation('relu'))
model.add(Dense(64, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(1,activation="linear", init='glorot_uniform', 
                W_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
#

sgd = SGD(lr=0.01, decay=0.01, momentum=0.9, nesterov=True)
rmsprop=RMSprop(lr=1e-5, rho=0.9, epsilon=1e-8)
model.name="deep_survival"

model.compile(loss=negative_log_likelihood(E_train), optimizer=rmsprop)



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_shape=(443,), kernel_initializer="glorot_uniform")`
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="glorot_uniform")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", activity_regularizer=<keras.reg..., kernel_initializer="glorot_uniform", kernel_regularizer=<keras.reg...)`


In [108]:
 pipeline.steps.append(['model', model])

In [109]:
pipeline

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ['model',
         verbose=False)

## MODEL FIT

In [111]:
print('Training...')
model.fit(X_train, Y_train,validation_split=0.33, batch_size=324, nb_epoch=400, shuffle=False, callbacks=[tensoboard])

hr_pred=model.predict(X_train)
hr_pred=np.exp(hr_pred)
ci=concordance_index(Y_train,-hr_pred,E_train)

hr_pred2=model.predict(X_val)
hr_pred2=np.exp(hr_pred2)
ci2=concordance_index(Y_val,-hr_pred2,E_val)
print ('Concordance Index for training dataset:', ci)
print ('Concordance Index for test dataset:', ci2)

#Cox Fitting
cf = CoxPHFitter()
cf.fit(rossi_dataset, 'week', event_col='arrest')

cf.print_summary()  # access the results using cf.summary

Training...
Train on 200 samples, validate on 100 samples


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/400
200/200 [==============================] - 0s 22us/step - loss: 0.4158 - val_loss: 0.0681
Epoch 2/400
200/200 [==============================] - 0s 21us/step - loss: 0.4244 - val_loss: 0.0682
Epoch 3/400
200/200 [==============================] - 0s 25us/step - loss: 0.3314 - val_loss: 0.0683
Epoch 4/400
200/200 [==============================] - 0s 25us/step - loss: 0.3353 - val_loss: 0.0683
Epoch 5/400
200/200 [==============================] - 0s 25us/step - loss: 0.4096 - val_loss: 0.0681
Epoch 6/400
200/200 [==============================] - 0s 24us/step - loss: 0.5759 - val_loss: 0.0681
Epoch 7/400
200/200 [==============================] - 0s 23us/step - loss: 0.5878 - val_loss: 0.0681
Epoch 8/400
200/200 [==============================] - 0s 25us/step - loss: 0.5900 - val_loss: 0.0681
Epoch 9/400
200/200 [==============================] - 0s 23us/step - loss: 0.4357 - val_loss: 0.0680
Epoch 10/400
200/200 [==============================] - 0s 23us/step - loss: 0.428

Epoch 81/400
200/200 [==============================] - 0s 20us/step - loss: 0.4688 - val_loss: 0.0663
Epoch 82/400
200/200 [==============================] - 0s 22us/step - loss: 0.5302 - val_loss: 0.0662
Epoch 83/400
200/200 [==============================] - 0s 23us/step - loss: 0.3212 - val_loss: 0.0663
Epoch 84/400
200/200 [==============================] - 0s 21us/step - loss: 0.4138 - val_loss: 0.0662
Epoch 85/400
200/200 [==============================] - 0s 23us/step - loss: 0.4140 - val_loss: 0.0661
Epoch 86/400
200/200 [==============================] - 0s 22us/step - loss: 0.2882 - val_loss: 0.0662
Epoch 87/400
200/200 [==============================] - 0s 24us/step - loss: 0.5855 - val_loss: 0.0661
Epoch 88/400
200/200 [==============================] - 0s 21us/step - loss: 0.4782 - val_loss: 0.0661
Epoch 89/400
200/200 [==============================] - 0s 22us/step - loss: 0.3080 - val_loss: 0.0661
Epoch 90/400
200/200 [==============================] - 0s 27us/step - lo

Epoch 160/400
200/200 [==============================] - 0s 24us/step - loss: 0.3880 - val_loss: 0.0642
Epoch 161/400
200/200 [==============================] - 0s 26us/step - loss: 0.2819 - val_loss: 0.0642
Epoch 162/400
200/200 [==============================] - 0s 24us/step - loss: 0.3804 - val_loss: 0.0642
Epoch 163/400
200/200 [==============================] - 0s 25us/step - loss: 0.2262 - val_loss: 0.0641
Epoch 164/400
200/200 [==============================] - 0s 26us/step - loss: 0.5210 - val_loss: 0.0642
Epoch 165/400
200/200 [==============================] - 0s 23us/step - loss: 0.2782 - val_loss: 0.0641
Epoch 166/400
200/200 [==============================] - 0s 24us/step - loss: 0.4771 - val_loss: 0.0640
Epoch 167/400
200/200 [==============================] - 0s 25us/step - loss: 0.5836 - val_loss: 0.0640
Epoch 168/400
200/200 [==============================] - 0s 21us/step - loss: 0.3172 - val_loss: 0.0640
Epoch 169/400
200/200 [==============================] - 0s 25us

Epoch 239/400
200/200 [==============================] - 0s 24us/step - loss: 0.2571 - val_loss: 0.0622
Epoch 240/400
200/200 [==============================] - 0s 26us/step - loss: 0.5115 - val_loss: 0.0622
Epoch 241/400
200/200 [==============================] - 0s 27us/step - loss: 0.2934 - val_loss: 0.0622
Epoch 242/400
200/200 [==============================] - 0s 32us/step - loss: 0.2643 - val_loss: 0.0624
Epoch 243/400
200/200 [==============================] - 0s 23us/step - loss: 0.4046 - val_loss: 0.0623
Epoch 244/400
200/200 [==============================] - 0s 20us/step - loss: 0.4573 - val_loss: 0.0622
Epoch 245/400
200/200 [==============================] - 0s 26us/step - loss: 0.4040 - val_loss: 0.0623
Epoch 246/400
200/200 [==============================] - 0s 29us/step - loss: 0.7285 - val_loss: 0.0621
Epoch 247/400
200/200 [==============================] - 0s 26us/step - loss: 0.2733 - val_loss: 0.0620
Epoch 248/400
200/200 [==============================] - 0s 26us

Epoch 318/400
200/200 [==============================] - 0s 21us/step - loss: 0.2721 - val_loss: 0.0605
Epoch 319/400
200/200 [==============================] - 0s 21us/step - loss: 0.2161 - val_loss: 0.0605
Epoch 320/400
200/200 [==============================] - 0s 21us/step - loss: 0.2528 - val_loss: 0.0605
Epoch 321/400
200/200 [==============================] - 0s 23us/step - loss: 0.2329 - val_loss: 0.0604
Epoch 322/400
200/200 [==============================] - 0s 22us/step - loss: 0.3057 - val_loss: 0.0604
Epoch 323/400
200/200 [==============================] - 0s 21us/step - loss: 0.4000 - val_loss: 0.0604
Epoch 324/400
200/200 [==============================] - 0s 28us/step - loss: 0.2889 - val_loss: 0.0602
Epoch 325/400
200/200 [==============================] - 0s 22us/step - loss: 0.4104 - val_loss: 0.0601
Epoch 326/400
200/200 [==============================] - 0s 24us/step - loss: 0.2344 - val_loss: 0.0600
Epoch 327/400
200/200 [==============================] - 0s 25us

Epoch 397/400
200/200 [==============================] - 0s 20us/step - loss: 0.2350 - val_loss: 0.0589
Epoch 398/400
200/200 [==============================] - 0s 25us/step - loss: 0.2820 - val_loss: 0.0588
Epoch 399/400
200/200 [==============================] - 0s 24us/step - loss: 0.2350 - val_loss: 0.0587
Epoch 400/400
200/200 [==============================] - 0s 28us/step - loss: 0.2562 - val_loss: 0.0587
Concordance Index for training dataset: 0.4547980981718342
Concordance Index for test dataset: 0.43823915900131405
